# RESNET-50

### TRANSFER LEARNING
1. Take layers from a previously trained model.
2. Freeze them, so as to avoid destroying any of the information they contain during future training rounds.
3. Add some new, trainable layers on top of the frozen layers. They will learn to turn the old features into predictions on a new dataset.
4. Train the new layers on your dataset.
5. Fine-tuning (optional), which consists of unfreezing the entire model you obtained above (or part of it), and re-training it on the new data with a very low learning rate

In [1]:
# CHECK INPUT IMAGE SHAPE
import cv2
input_path = r'C:\Users\User\Documents\UKM\Project\Dataset\Consolidated Dataset\organized'
inp = input_path + '\casme2-sub01_EP02_01f-happiness.jpg'
img = cv2.imread(inp)
print(img.shape)

(224, 224, 3)


In [ ]:
# CREATE TRAINING AND VALIDATION DATASET FROM MICRO_EXPRESION COMBINED DATASET
from tensorflow.keras.preprocessing import image_dataset_from_directory

ME_path = r"C:\Users\User\Documents\UKM\Project\Dataset\Consolidated Dataset\organized"
me_train_dataset = image_dataset_from_directory(ME_path, labels='inferred', label_mode='int',
                    validation_split=0.2, subset='training',
                    seed=1234, image_size=(224,224), color_mode='rgb')
me_val_dataset = image_dataset_from_directory(ME_path, labels='inferred', label_mode='int',
                    validation_split=0.2, subset='validation',
                    seed=1234, image_size=(224,224), color_mode='rgb')

print(me_train_dataset.class_names)

Found 333 files belonging to 7 classes.
Using 267 files for training.
Found 333 files belonging to 7 classes.
Using 66 files for validation.
['anger', 'contempt', 'disgust', 'fear', 'happiness', 'sadness', 'surprise']


In [74]:
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras import Input, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D


In [75]:
# BUILD THE BASE MODEL - RESNET50
# do not include the FC layer at the top of network
# and freeze the base model for transfer learning
inputShape = (224,224,3)
resnet50_model = ResNet50(weights='imagenet', include_top=False, input_shape=inputShape)
resnet50_model.trainable = False #freeze the base model
resnet50_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_12[0][0]                   
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

In [68]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [78]:
# ADD NEW TRAINABLE LAYERS ON TOP TO BUILD THE FINAL MODEL
from tensorflow.keras.layers.experimental.preprocessing import RandomContrast, RandomFlip

inputs = Input(shape=(224,224,3))
data_augmentation = Sequential([RandomFlip('horizontal'), RandomContrast(0.2)])

x = preprocess_input(inputs)
x = resnet50_model(x, training=False) #run in inference mode
x = GlobalAveragePooling2D()(x)
outputs = Dense(7,activation='softmax')(x) #7 output classes
model = Model(inputs, outputs)

In [82]:
# COMPILE THE NEW MODEL
base_learning_rate = 0.0001
model.compile(optimizer= Adam(learning_rate=base_learning_rate),
              loss = SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])
model.summary()
print(len(model.trainable_variables))

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf.__operators__.getitem_4 ( (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add_4 (TFOpLambda (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_5 ( (None, 2048)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 14343     
Total params: 23,602,055
Trainable params: 14,343
Non-trainable params: 23,587,712
__________________________________________

In [83]:
# TRAIN NEW MODEL ON MICRO-EXPRESSION DATASET
# check initial accuracy before training
loss0, accuracy0 = model.evaluate(me_val_dataset)
print('Initial loss: {} accuracy: {}'.format(loss0, accuracy0))

# train on new (Macro-expression) dataset and print accuracy after training
history = model.fit(me_train_dataset, epochs=100, validation_data= me_val_dataset)

3/3 [==============================] - 12s 2s/step - loss: 1.4577 - accuracy: 0.4545
Initial loss: 1.4577338695526123 accuracy: 0.4545454680919647
Epoch 1/100
9/9 [==============================] - 57s 6s/step - loss: 1.1468 - accuracy: 0.5955 - val_loss: 1.4461 - val_accuracy: 0.4545
Epoch 2/100
9/9 [==============================] - 54s 6s/step - loss: 1.1255 - accuracy: 0.6067 - val_loss: 1.4551 - val_accuracy: 0.4545
Epoch 3/100
9/9 [==============================] - 56s 6s/step - loss: 1.1194 - accuracy: 0.6142 - val_loss: 1.4593 - val_accuracy: 0.4394
Epoch 4/100
9/9 [==============================] - 59s 7s/step - loss: 1.1123 - accuracy: 0.6142 - val_loss: 1.4572 - val_accuracy: 0.4848
Epoch 5/100
9/9 [==============================] - 68s 8s/step - loss: 1.1065 - accuracy: 0.6067 - val_loss: 1.4604 - val_accuracy: 0.4697
Epoch 6/100
9/9 [==============================] - 59s 7s/step - loss: 1.0980 - accuracy: 0.6105 - val_loss: 1.4547 - val_accuracy: 0.4394
Epoch 7/100
9/9 [==

In [84]:
# MODEL IS OVERFITTING (TRAINING ACCURACY GETTING HIGHER WHILE VALIDATION ACCURACY PLATEAUED)

from tensorflow.keras.layers.experimental.preprocessing import RandomContrast, RandomFlip

inputs = Input(shape=(224,224,3))
data_augmentation = Sequential([RandomFlip('horizontal'), RandomContrast(0.2)])

x = data_augmentation(inputs) #add data augmentation step
x = preprocess_input(inputs)
x = resnet50_model(x, training=False) #run in inference mode
x = GlobalAveragePooling2D()(x)
outputs = Dense(7,activation='softmax')(x) #7 output classes
model2 = Model(inputs, outputs)

base_learning_rate = 0.00001 #reduce LR
model2.compile(optimizer= Adam(learning_rate=base_learning_rate),
              loss = SparseCategoricalCrossentropy(),
              metrics = ['accuracy'])
history2 = model2.fit(me_train_dataset, epochs=20, validation_data= me_val_dataset)

Epoch 1/20
9/9 [==============================] - 59s 6s/step - loss: 2.1665 - accuracy: 0.1873 - val_loss: 2.1302 - val_accuracy: 0.2121
Epoch 2/20
9/9 [==============================] - 58s 7s/step - loss: 2.1389 - accuracy: 0.1873 - val_loss: 2.1089 - val_accuracy: 0.1970
Epoch 3/20
9/9 [==============================] - 57s 6s/step - loss: 2.1147 - accuracy: 0.1873 - val_loss: 2.0897 - val_accuracy: 0.1970
Epoch 4/20
9/9 [==============================] - 65s 7s/step - loss: 2.0904 - accuracy: 0.1910 - val_loss: 2.0728 - val_accuracy: 0.2121
Epoch 5/20
9/9 [==============================] - 67s 7s/step - loss: 2.0693 - accuracy: 0.1873 - val_loss: 2.0564 - val_accuracy: 0.2121
Epoch 6/20
9/9 [==============================] - 61s 7s/step - loss: 2.0498 - accuracy: 0.1873 - val_loss: 2.0415 - val_accuracy: 0.1970
Epoch 7/20
9/9 [==============================] - 58s 6s/step - loss: 2.0313 - accuracy: 0.1873 - val_loss: 2.0284 - val_accuracy: 0.1970
Epoch 8/20
9/9 [==================